# Compte rendu du TP 1 SD-TSIA 211
# Recommender System

In [1]:
import numpy as np
import sys
from scipy import sparse
from scipy.optimize import check_grad
from numpy import linalg as LA
from scipy.sparse.linalg import svds
from timeit import default_timer as timer

### 1)  Présentation du modèle :

#### Question 1.1


In [2]:
def load_movielens(filename, minidata=False):
    """
    Cette fonction lit le fichier filename de la base de donnees
    Movielens, par exemple 
    filename = '~/datasets/ml-100k/u.data'
    Elle retourne 
    R : une matrice utilisateur-item contenant les scores
    mask : une matrice valant 1 si il y a un score et 0 sinon
    """

    data = np.loadtxt(filename, dtype=int)

    R = sparse.coo_matrix((data[:, 2], (data[:, 0] - 1, data[:, 1] - 1)),
                          dtype=float)
    R = R.toarray()  # not optimized for big data

    # code la fonction 1_K
    mask = sparse.coo_matrix((np.ones(data[:, 2].shape),
                              (data[:, 0] - 1, data[:, 1] - 1)), dtype=bool)
    mask = mask.toarray()  # not optimized for big data

    if minidata is True:
        R = R[0:100, 0:200].copy()
        mask = mask[0:100, 0:200].copy()

    return R, mask

In [3]:
filename = "u.data"
R, mask = load_movielens(filename)
n, k = R.shape
print(n, k)

943 1682


On obtient effectivement une matrice de dimensions 943 x 1682.
L'option mindata si mise à True retourne uniquement une partie des données de dimensions 100 x 200 pour R et mask.

#### Question 1.2


In [4]:
print("Le nombre d'utilisateur est : " + str(n))
print("Le nombre total de notes est : " + str(sum(sum(mask))))

Le nombre d'utilisateur est : 943
Le nombre total de notes est : 100000


Il existe 943 utilisateurs et 1682 films référencés dans la base de données.
Le nombre total de notes = 100 000.

#### Question 1.3 : 

   <strong> La fonction objectif n'est pas convexe : </strong>
On prend le cas scalaire où $\rho = 0 $
la fontion objectif devient alors : $f(p,q) = \frac{1}{2}*(r-pq)^2 $<br\>
On obtient alors: $\nabla^2 f(p,q) =  \begin{bmatrix} q^2 & 2qp-r \\ 2qp-r & p^2 \end{bmatrix} $
; Pour $ x = \begin{bmatrix}p\\q\end{bmatrix}  $ on a <br\>$x^T\nabla^2 f(p,q) x = 2pq(3pq-r) $ tel que pour $ q = p = \frac{\sqrt(|r|)}{\sqrt(5)}$ on aura $x^T\nabla^2 f(p,q) x = -\frac{4}{25}r² < 0  $ <br/>
Alors $\nabla^2 f$ n'est pas positive dans ce cas donc on peut généraliser ce cas et dire que la fonction objectif n'est pas convexe en (P,Q)

<strong> Calculer le gradient de la fonction objectif </strong>

On a : $g(P,Q)=\frac{1}{2} \|\mathcal{1_K \circ (R - QP)}\|^2_F+\rho\|\mathcal{Q}\|^2_F+\rho\|\mathcal{P}\|^2_F $
<br\>Alors : $\nabla_Q f(P,Q) = \rho Q - (\mathcal{1_K \circ (R-QP)}\ ) P^T $ <br\> et 
$\nabla_P f(P,Q) = \rho P - Q^T(\mathcal{1_K \circ (R-QP)}\ )  $

### 2) Trouver P quand $Q_0$ est fixé :

#### Question 2.1:
Le gradient de la fonction objectif devient alors : $\nabla g(P) = \rho P - {Q^0}^T(\mathbb{1}_K \circ (R - Q^0P))$ <br\>
$ \nabla^{2} g(P) =  \mathbb{1}_K\circ(Q^{0})^{T}Q^{0} + \rho Id $ or $\rho = 0.3 > 0 $  , alors $\nabla^{2} g(P)$ est définie positive <br/> alors la fonction objectif à Q fixé est convexe <br\>
$ L_0 = \rho + \|Q_0^T Q_0\|_F $

#### Question 2.2


In [5]:
def objective(P, Q0, R, mask, rho):
    """
    La fonction objectif du probleme simplifie.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q0 : une matrice de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction en (P,Q0)
    grad_P : le gradient par rapport a P
    """

    tmp = (R - Q0.dot(P)) * mask

    val = np.sum(tmp ** 2) / 2. + rho / 2. * (np.sum(Q0 ** 2) + np.sum(P ** 2))

    grad_P = rho * P - np.dot(np.transpose(Q0), tmp)

    return val, grad_P

In [6]:
c = 4
epsi = 700
rho = 0.3
Q0, singval, P0 = svds(R, k=c)

In [7]:
def func(X):
    S = X.reshape(c, k)
    func, grad = objective(S, Q0, R, mask, rho)
    return func


def grad(X):
    S = X.reshape(c, k)
    func, grad = objective(S, Q0, R, mask, rho)
    return np.ravel(objective)

#print(check_grad(func, grad, np.ravel(P0)))

#### Question 2.3:

In [8]:
def Gradient(g, P0, gamma, epsilon):
    '''
    Calcule le gradient et le point minimal de la fonction g
    Entrées :
    g : foncrion objectif
    P : la variable matricielle de taille C x I
    gamma : pas de la méthode de descente
    epsilon : condition d'arrêt

    Sorties :
    val : le gradient de g 
    x_k : le min de la fonction g 
    '''
    x_k = P0
    val, grad = g(P0)
    while(np.linalg.norm(grad, ord='fro') > epsilon):
        x_k = x_k - gamma * grad
        val, grad = g(x_k)
    return val, x_k

#### Question 2.4

In [9]:
epsilon = 1
L0 = rho + np.linalg.norm(np.transpose(Q0).dot(Q0), ord='fro')
gamma = 1 / L0


def g(P): return objective(P, Q0, R, mask, rho)


val, m = Gradient(g, P0, gamma, epsilon)
print(val)

369551.549915


## Raffinements algorithmiques pour le problème à $Q_0$ fixé

## question 3.1


In [10]:
def armijo(g, P, a, b, beta):
    '''
    La fonction de recherche linéaire d'Armijo pour déterminer b*a^l a.k.a gamma
    prend en entrée : 
    g : la fonction objectif
    P : la variable matricielle de taille C x I
    a : tel que a dans [0,1]
    b : tel que b > 0
    beta : tel que beta dans [0,1]

    Sortie 
    gamma : le pas de de la méthode du gradient descendant
    '''
    gP, gradP = g(P)  # Récupérer le gradient de la fonction objectif en P

    gamma = b
    x = P - gamma * gradP
    gx, gradx = g(x)
    while (gx > (gP + beta * np.sum(gradP * (x - P)))):
        gamma = gamma * a
        x = P - gamma * gradP
        gx, gradx = g(x)

    return gamma

In [11]:
def Gradient_Line_Search(g, P0, gamma0, epsilon):
    '''
    La fonction retourne le gradient de g et un minimiseur de g 
    Entrées:
    g : fonction objectif
    P0 : La variable matricielle de taille C x I
    epsilon : facteur de précision
    gamma0 : pas initial de la méthode Gradient Linear Search

    Sorties:
    gradP : le gradient de g
    val : min de g
    '''
    [val, gradP] = objective(P0, Q0, R, mask, rho)
    while(np.linalg.norm(gradP, ord='fro') > epsilon):

        P0 = P0 - gamma0 * gradP
        gamma0 = armijo(g, P0, 0.5, 2 * gamma0, 0.5)
        [val, gradP] = g(P0)

    return (val, gradP)

In [12]:
val_ls, gradP_ls = Gradient_Line_Search(g, P0, 0.5, 1)

In [13]:
print("La valeur obtenue par la méthode armijo's line search  est " + str(val_ls))

La valeur obtenue par la méthode armijo's line search  est 369551.402967


#### Question 3.2

$g(P) = \frac{1}{2}\|\mathbb{1}_{K}\circ(R-Q^{0}P)\|^2_F + \frac{\rho}{2}\|Q^{0}\|^2_F + \frac{\rho}{2}\|P\|^2_F$
  
$
= \frac{1}{2}P^T\left((Q^0)^{T}(\mathbb{1}_{K}\circ(\mathbb{1}_{K})^{T})Q^0 + \rho Id\right)P - \frac{1}{2}((\mathbb{1}_{K}\circ(R))^{T}Q^0P + P^{T}(Q^0)^{T}(\mathbb{1}_{K}\circ(R))) + \frac{1}{2}((\mathbb{1}_{K}\circ(R))^{T}(\mathbb{1}_{K}\circ(R)) + \rho(Q^0)^{T}Q^0) \\
$

or on a :   
$\begin{align*}
(\mathbb{1}_{K}\circ(R))^TQ^0P &= \langle (\mathbb{1}_{K}\circ(R)), Q^0P \rangle \\
&= \langle Q^0P, (\mathbb{1}_{K}\circ(R)) \rangle \\
&= P^{T}(Q^0)^{T}(\mathbb{1}_{K}\circ(R))
\end{align*}$  

donc:
$g(P) = \frac{1}{2}P^T\left(\mathbb{1d}_{K}\circ(Q^0)^TQ^0 + \rho Id\right)P - \mathbb{1d}_K\circ R^TQ^0P + \frac{1}{2}\left(\mathbb{1d}_K\circ R^TR + \rho(Q^0)^TQ^0\right)$

qui est de la forme : $\frac{1}{2}P^TAP + BP + C$ 

avec $A = \left(\mathbb{1d}_{K}\circ(Q^0)^TQ^0 + \rho Id\right)$ <br\>
$B = -\mathbb{1d}_K\circ R^TQ^0 $<br\> 
$C = \frac{1}{2}\left(\mathbb{1d}_K\circ R^TR + \rho(Q^0)^TQ^0\right)$

<br\> Comme $\rho>0$: alors la matrice $A$ est définie positive.


<br\>On utilise donc la méthode du gradient conjugué.

In [14]:
def GradientConjuge(g, P0, Q0, epsilon):
    '''
    La fonction retourne le gradient de g et un minimiseur de g en appliquant la méthode du gradient conjugué
    Entrées:
    g : fonction objectif
    P0 : La variable matricielle de taille C x I
    Q0 : La variable matricielle de taille U x C
    epsilon : Facteur de précision

    Sorties:
    P : le gradient de g
    g(P)[0] : min de g
    '''
    grad_gP = g(P0)[1]
    direction = - grad_gP
    prod = rho * direction + np.transpose(Q0).dot(mask * (Q0.dot(direction)))
    # la matrice A fois d
    s = - np.sum(direction * grad_gP) / np.sum(direction * prod)
    P = P0 + s * direction
    while(np.linalg.norm(grad_gP) > epsilon):
        grad_gP = g(P)[1]
        b = np.sum(grad_gP * prod) / np.sum(direction * prod)
        direction = - grad_gP + b * direction
        prod = rho * direction + Q0.T.dot(mask * (Q0.dot(direction)))
        s = - np.sum(direction * grad_gP) / np.sum(direction * prod)
        P = P + s * direction
    return P, g(P)[0]

In [15]:
p, val = GradientConjuge(g, P0, Q0, 1)
print(val)

369550.62773


#### Question 3.3

In [16]:
start = timer()
[val_opt, P_opt] = Gradient(g, P0, gamma, epsilon)
end = timer()
grad_pas_constant = end - start

start = timer()
val_ls, gradP_ls = Gradient_Line_Search(g, P0, 0.5, 1)
end = timer()
gradient_ls = end - start


start = timer()
p, val = GradientConjuge(g, P0, Q0, 1)
end = timer()
gradient_conj = end - start
print("Le temps d'excecution de gradient pas constant = " + str(grad_pas_constant))
print("Le temps d'excecution de gradient line search est = " + str(gradient_ls))
print("Le temps d'excecution de gradient conjugué est = " + str(gradient_conj))

Le temps d'excecution de gradient pas constant = 0.672619836001104
Le temps d'excecution de gradient line search est = 0.7108894660013902
Le temps d'excecution de gradient conjugué est = 0.2825142660003621


## Résolution du problème complet

#### Question 4.1

In [17]:
def total_objective(P, Q, R, mask, rho):
    """
    La fonction objectif du probleme complet.
    Prend en entree 
    P : la variable matricielle de taille C x I
    Q : la variable matricielle de taille U x C
    R : une matrice de taille U x I
    mask : une matrice 0-1 de taille U x I
    rho : un reel positif ou nul

    Sorties :
    val : la valeur de la fonction
    grad_P : le gradient par rapport a P
    grad_Q : le gradient par rapport a Q
    """

    tmp = (R - Q.dot(P)) * mask

    val = np.sum(tmp ** 2) / 2. + rho / 2. * (np.sum(Q ** 2) + np.sum(P ** 2))

    grad_P = rho * P - Q.T.dot(tmp)

    grad_Q = rho * Q - tmp.dot(P.T)
    return val, grad_P, grad_Q

In [18]:
def ArmijoGeneralized(g, P, Q, a, b, beta):
    '''
    La fonction de recherche linéaire d'Armijo pour déterminer b*a^l a.k.a gamma
    prend en entrée : 
    g : la fonction objectif
    P : la variable matricielle de taille C x I
    Q : la variable matricielle de taille U x C
    a : tel que a dans [0,1]
    b : tel que b > 0
    beta : tel que beta dans [0,1]

    Sortie 
    gamma : le pas de de la méthode du gradient descendant
    '''
    val, grad_P, grad_Q = g(P, Q)
    gamma = b
    P_plus = P - gamma * grad_P
    Q_plus = Q - gamma * grad_Q
    while (g(P_plus, Q_plus)[0] > val + beta * np.sum(grad_P * (P_plus - P)) + beta * np.sum(grad_Q * (Q_plus - Q))):
        gamma *= a
        P_plus = P - gamma * grad_P
        Q_plus = Q - gamma * grad_Q

    return gamma

In [19]:
def Gradient_Line_Search_Generalized(g, P0, Q0, gamma0, epsilon):
    '''
    Retourne le min de la fonction objectif généralisée 
    Entrées:
    g_gen : fonction objectif généralisée
    P0 : la variable matricielle de taille C x I
    Q0 : la variable matricielle de taille U x C
    gamma0 : Pas initial
    epsilon : facteur de précision
    Sortie:
    val : valeur minimale
    '''
    P = P0
    Q = Q0
    val, grad_P, grad_Q = g(P, Q)
    gamma = ArmijoGeneralized(g, P, Q, 0.5, 2 * gamma0, 0.5)

    while (np.linalg.norm(grad_P) > epsilon or np.linalg.norm(grad_Q) > epsilon):
        P = P - gamma * grad_P
        Q = Q - gamma * grad_Q
        gamma = ArmijoGeneralized(g, P, Q, 0.5, 2 * gamma, 0.5)
        val, grad_P, grad_Q = g(P, Q)

    return P, Q, val

In [20]:
epsilon=100
gamma0 = 1
t1=timer()
P, Q, val = Gradient_Line_Search_Generalized(lambda P,Q : total_objective(P, Q, R, mask, rho), P0, Q0, gamma0, epsi)
t2=timer()
P,Q,val,t2-t1

(array([[-0.26620423,  0.02080478,  0.65317844, ..., -0.04728094,
         -0.00551023,  0.07793016],
        [ 0.38685675, -0.70719179, -0.15535588, ...,  0.05141169,
         -0.01844841,  0.00492953],
        [-1.01435488,  0.00851166, -0.82183963, ..., -0.05219049,
          0.01750809,  0.02657223],
        [ 2.02208148,  1.53540481,  1.42863097, ...,  0.02696574,
          0.13826988,  0.14081172]]),
 array([[ 0.80045223,  0.31817446,  0.14528985,  2.12185149],
        [ 0.065585  ,  0.94742262, -0.87234076,  1.17450003],
        [-0.35266131,  0.60695575, -0.62291355,  0.96416904],
        ..., 
        [ 0.30694508,  0.31333343, -0.99334199,  1.40814168],
        [-0.92097246,  0.17282727,  0.40038025,  2.14151306],
        [ 0.72988517, -0.61061147, -0.35498355,  1.93663952]]),
 44814.266408267606,
 2.8758017210020625)

#### Question 4.2:



Soit $k \in \mathbb{N}^*$.

On a  : pour tout $ P, \quad g(P_{k+1}, Q_k) \leq g(P, Q_k)$ donc :$ g(P_{k+1}, Q_k) \leq g(P_k, Q_k)$


et on a : pour tout $ Q, \quad g(P_{k+1}, Q_{k+1}) \leq g(P_{k+1}, Q) $
donc : $ g(P_{k+1}, Q_{k+1}) \leq g(P_{k+1}, Q_k) $

finalement: pour tout $k \in \mathbb{N}^*, \quad g(P_{k+1}, Q_{k+1}) \leq g(P_k, Q_k)$

Conclusion : $(g(P_k,Q_k))$ $ k\in\mathbb{N}^* $ est décroissante.
Et puisque $ 0 \leq g(P_k,Q_k)$ alors la suite $(g(P_k,Q_k))$ est décroissante et minorée donc elle converge.

#### Question 4.3


In [21]:
def objective_Q(P0, Q, R, mask, rho):

    tmp = (R - Q.dot(P0)) * mask

    val = np.sum(tmp ** 2) / 2. + rho / 2. * (np.sum(Q ** 2) + np.sum(P0 ** 2))

    grad_Q = rho * Q - np.dot(tmp, np.transpose(P0))

    return val, grad_Q

def gradientLineSearch(g, P0, gamma0, epsilon):
    
    P = P0
    gP, grad_gP = g(P)
    gamma = armijo(g, P, 0.5, 2*gamma0, 0.5)
    
    while (np.linalg.norm(grad_gP) > epsilon):
        P = P - gamma * grad_gP
        gamma = armijo(g, P, 0.5, 2*gamma, 0.5)
        gP, grad_gP = g(P)
    
    return P, gP  

def LS_Alt(P0, Q0, R, mask, rho, gamma, epsilon):
    P = P0
    Q = Q0
    val, gradP, gradQ = total_objective(P, Q, R, mask, rho)
    while (np.linalg.norm(gradP) > epsilon) or (np.linalg.norm(gradQ) > epsilon):
        P, val = gradientLineSearch(lambda P: objective(
            P, Q, R, mask, rho), P, gamma, epsilon)

        Q, val = gradientLineSearch(lambda Q: objective_Q(
            P, Q, R, mask, rho), Q,  gamma, epsilon)

        val, gradP, gradQ = total_objective(P, Q, R, mask, rho)


    return P, Q, val

#### Question 4.4 :

In [22]:
epsilon = 100
t1=timer()
minLS_P, minLS_Q, val = LS_Alt(P0, Q0, R, mask, rho, gamma, epsi)
t2=timer()
minLS_P,minLS_Q,val,t2-t1

(array([[  2.23804388e+00,   4.49704424e-01,   6.77817742e+00, ...,
          -1.32622117e-01,   1.07596505e-02,   2.43428224e-01],
        [ -3.20717528e+00,  -1.18641511e+01,  -2.21042079e+00, ...,
           1.01359068e-01,  -8.64003283e-02,  -4.97074436e-02],
        [ -1.85841932e+01,  -1.49497760e+00,  -6.10605290e+00, ...,
          -9.57335320e-02,   2.24813597e-02,   4.33986589e-02],
        [  5.34530804e+01,   1.95997311e+01,   1.11053104e+01, ...,
           1.69422898e-02,   1.84664476e-01,   1.76733021e-01]]),
 array([[ 0.08074965,  0.04744437,  0.01355601,  0.11908643],
        [ 0.03060962,  0.13622213, -0.06445243,  0.0510773 ],
        [-0.03420024,  0.13393793,  0.00036797,  0.05998684],
        ..., 
        [ 0.07428433,  0.0557645 , -0.06364925,  0.07305806],
        [-0.18829466,  0.04628578,  0.05410047,  0.09841898],
        [ 0.12224065, -0.09809558, -0.01099657,  0.09028144]]),
 234300.14411245758,
 16.293220187999395)

#### Question 4.5

In [23]:
R1=Q.dot(P)[300]
score = max(R1)
score

4.7224195178620176

Retrouvons le film à recommender:

In [24]:
np.argmax(R1)+1

64

le film à recommender est le film d'id 64